# Mini Türkçe Transformer Modeli - V1

Bu çalışma sıfırdan yazılmış basit bir Transformer Decoder modelidir.

Amaç:
- Tokenizer mantığını anlamak
- Embedding ve Attention yapısını görmek
- Küçük veri üzerinde dil modeli eğitmek

Not:
Bu model eğitim amaçlıdır, üretim için optimize edilmemiştir.

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset

## VOCAB

In [3]:
text = """
malezya havayollarına ait mh370 sefer sayılı yolcu uçağı 8 mart 2014 tarihinde kayboldu.
uçak kuala lumpur şehrinden pekin şehrine doğru uçuyordu.
uçakta iki yüz otuz dokuz yolcu ve mürettebat bulunuyordu.

uçak kalkıştan kısa bir süre sonra hava trafik kontrolü ile iletişimini kaybetti.
transponder sistemi kapandı ve uçak radarlardan kayboldu.
askeri radar verilerine göre uçak rotasını değiştirdi ve batıya doğru yöneldi.

uydu verileri uçağın saatlerce havada kaldığını gösterdi.
ancak kesin konumu belirlenemedi.
arama çalışmaları hint okyanusu üzerinde yoğunlaştı.

birçok ülke arama çalışmalarına katıldı.
avustralya çin ve malezya ekipleri geniş çaplı operasyonlar yürüttü.
deniz tabanı sonar sistemleri ile tarandı.

2015 yılında hint okyanusunda bir kanat parçası bulundu.
uzmanlar bu parçanın mh370 uçağına ait olduğunu doğruladı.
daha sonra mozambik ve madagaskar kıyılarında da bazı enkaz parçaları bulundu.

ancak uçağın ana gövdesi ve kara kutusu hala bulunamadı.
bu durum olayın gizemini artırdı.
uzmanlar farklı teoriler ortaya attı.

bazı teorilere göre uçak bilinçli olarak rotasını değiştirdi.
bazı araştırmacılar teknik bir arıza ihtimali üzerinde durdu.
bazı kişiler ise pilot müdahalesi olabileceğini iddia etti.

resmi soruşturmalar kesin bir sonuca ulaşamadı.
olay modern havacılık tarihinin en büyük gizemlerinden biri olarak kabul edilmektedir.
mh370 vakası havacılık güvenliği konusunda yeni düzenlemelere yol açtı.
uçak takip sistemlerinin geliştirilmesi gerektiği vurgulandı.
"""

tokens_raw = text.replace(".", " .").split()
unique_tokens = sorted(set(tokens_raw))

vocab = {token: idx for idx, token in enumerate(unique_tokens)}

# özel tokenlar
vocab[" "] = len(vocab)
vocab["<unk>"] = len(vocab)

print("Vocab size:", len(vocab))
print(vocab)

Vocab size: 163
{'.': 0, '2014': 1, '2015': 2, '8': 3, 'ait': 4, 'ana': 5, 'ancak': 6, 'arama': 7, 'araştırmacılar': 8, 'artırdı': 9, 'arıza': 10, 'askeri': 11, 'attı': 12, 'avustralya': 13, 'açtı': 14, 'batıya': 15, 'bazı': 16, 'belirlenemedi': 17, 'bilinçli': 18, 'bir': 19, 'biri': 20, 'birçok': 21, 'bu': 22, 'bulunamadı': 23, 'bulundu': 24, 'bulunuyordu': 25, 'büyük': 26, 'da': 27, 'daha': 28, 'deniz': 29, 'değiştirdi': 30, 'dokuz': 31, 'doğru': 32, 'doğruladı': 33, 'durdu': 34, 'durum': 35, 'düzenlemelere': 36, 'edilmektedir': 37, 'ekipleri': 38, 'en': 39, 'enkaz': 40, 'etti': 41, 'farklı': 42, 'geliştirilmesi': 43, 'geniş': 44, 'gerektiği': 45, 'gizemini': 46, 'gizemlerinden': 47, 'göre': 48, 'gösterdi': 49, 'gövdesi': 50, 'güvenliği': 51, 'hala': 52, 'hava': 53, 'havacılık': 54, 'havada': 55, 'havayollarına': 56, 'hint': 57, 'iddia': 58, 'ihtimali': 59, 'iki': 60, 'ile': 61, 'iletişimini': 62, 'ise': 63, 'kabul': 64, 'kaldığını': 65, 'kalkıştan': 66, 'kanat': 67, 'kapandı': 68, '

## 1 - TOKENIZER

In [5]:
class Tokenizer:
  def __init__(self, vocab_json):
    self.vocab = vocab_json
    self.reverse_vocab = {v: k for k, v in self.vocab.items()}

  def encode(self, text):
    tokens = []
    for word in text.split():
      i = 0
      while i < len(word):
        found_match = False
        for j in range(len(word), i, -1):
          subword = word[i:j]
          if subword in self.vocab:
            tokens.append(self.vocab[subword])
            found_match = True
            i=j
            break
        if not found_match:
          tokens.append(self.vocab['<unk>'])
          i += 1
      tokens.append(self.vocab[" "])
    tokens.pop()
    return tokens

  def decode(self, ids):
    text = ""
    for id in ids:
      text += self.reverse_vocab[id]
    return text
  
  def tokenize(self, text):
    ids = self.encode(text)
    words = [self.reverse_vocab[id] for id in ids]
    return words

## EMBEDDING

In [6]:
def get_rotary_position_encoding(input: torch.Tensor, base=10000, device="cpu"):
    context_length, embedding_dim = input.shape
    assert embedding_dim % 2 == 0, "Embedding dimension must be even for rotary position encoding."
    half_dim = embedding_dim // 2
    
    freq_indices = torch.arange(half_dim, device=device, dtype=torch.float32)
    freqs = 1.0 / (base ** (freq_indices / half_dim))
    
    positions = torch.arange(context_length, device=device, dtype=torch.float32).unsqueeze(1)
    
    angles = positions * freqs
    sin_angles = torch.sin(angles)
    cos_angles = torch.cos(angles)
    
    input_even = input[:, :half_dim]
    input_odd = input[:, half_dim:]
    
    rotated_input_even = input_even * cos_angles - input_odd * sin_angles
    rotated_input_odd = input_even * sin_angles + input_odd * cos_angles
    
    input_rotated = torch.zeros_like(input)
    input_rotated[:, :half_dim] = rotated_input_even
    input_rotated[:, half_dim:] = rotated_input_odd
    
    return input_rotated

In [7]:
class Embedding(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.pos_embedding = get_rotary_position_encoding

    def forward(self, input_ids):
        x = self.embedding(input_ids) # dictory'deki kelimelerin vektör temsilleri
        x = self.pos_embedding(x) # pozisyonlarına göre döndürülmüş vektör temsilleri
        return x

## MULTI-HEAD ATTENTION

In [8]:
class MultiHeadAttention(nn.Module):
    def __init__(self, embedding_dim, output_dim, context_length, num_heads=2, dropout_rate=0):
        super().__init__()
        
        self.context_length = context_length
        self.multi_head_attention = nn.MultiheadAttention(embedding_dim, num_heads, dropout=dropout_rate)
        self.projection = nn.Linear(embedding_dim, output_dim)
        
        self.register_buffer("mask", torch.tril(torch.ones(context_length, context_length)))  # Maksimum context length için mask
    
    def forward(self, x):
        number_of_tokens = x.shape[0]
        x = x[:self.context_length]
        x = self.multi_head_attention(x, x, x, attn_mask=self.mask[:number_of_tokens, :number_of_tokens])[0]
        x = self.projection(x)
        return x

## LAYER NORMALIZATION

In [9]:
class LayerNorm(nn.Module):
    def __init__(self, embedding_dim, eps=1e-5):
        super().__init__()
        
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(embedding_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        normalized = (x - mean) / torch.sqrt(var + self.eps)
        return self.weight * normalized

## ACTIVATION FUNCTION - GELU

In [10]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(torch.sqrt(2 / torch.tensor(torch.pi)) * (x + 0.044715 * torch.pow(x, 3))))

## MLP (MULTI-LAYER PERCEPTRON)

In [11]:
class MLP(nn.Module):
    def __init__(self, embedding_dim, hidden_dim):
        super().__init__()
        
        self.gate_proj = nn.Linear(embedding_dim, hidden_dim)
        self.up_proj = nn.Linear(embedding_dim, hidden_dim)
        self.down_proj = nn.Linear(hidden_dim, embedding_dim)
        
        self.gelu = GELU()

    def forward(self, x):
        gate = self.gate_proj(x)
        gate = self.gelu(gate)
        up = self.up_proj(x)
        fuse = gate * up
        outputs = self.down_proj(fuse)
        return outputs

## DECODER BLOCK

In [12]:
class DecoderBlock(nn.Module):
    def __init__(self, embedding_dim, context_length, num_heads=2, dropout_rate=0.5):
        super().__init__()
        
        self.self_attention = MultiHeadAttention(embedding_dim, embedding_dim, context_length, num_heads=num_heads, dropout_rate=dropout_rate)
        self.norm1 = LayerNorm(embedding_dim)
        self.mlp = MLP(embedding_dim, embedding_dim)
        self.norm2 = LayerNorm(embedding_dim)

    def forward(self, x):
        """
          |------> Norm Block 1 (1) --------------------------------------------> |                                 |----------> Norm Block 2 ------------------------> |
        x |                                                                       |----> + (First connection)-----> |                                                   |------> + (Second connection)
          |------> Multi-head Attention Block (2) -----> Norm Block 1 (3) ------> |                                 |----------> MLP Block -----> Norm Block 2 -------> |
        """
        # First Connection
        normalized_x = self.norm1(x) # (1) Normalization
        
        attention_x = self.self_attention(x) # (2) Multi-head Attention
        post_attention_x = self.norm1(attention_x) # (3) Normalization after attention
        
        x = normalized_x + post_attention_x # Residual connection after attention
        
        # Second Connection
        normalized_x = self.norm2(x) 
        
        mlp_x = self.mlp(x) 
        post_attention_x = self.norm2(mlp_x) 
        
        x = normalized_x + post_attention_x 
        return x

## 2- MODEL

In [13]:
class MasterLLMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_length, dropout_rate=0.5, num_heads=2, num_layers=2):
        super().__init__()
        
        self.embedding = Embedding(vocab_size=vocab_size, embedding_dim=embedding_dim)
        
        self.layers = nn.Sequential(
            *[DecoderBlock(embedding_dim, context_length, num_heads=num_heads, dropout_rate=dropout_rate) for _ in range(num_layers)]
            )
        
        self.lm_head = nn.Linear(embedding_dim, vocab_size)

    def forward(self, input_ids):
        x = self.embedding(input_ids) # dictory'deki kelimelerin vektör temsilleri
        x = self.layers(x)
        x = self.lm_head(x)
        return x
    
    def generate(self, x, max_new_tokens):
        tokens = x
        x = torch.tensor(x)
        
        for _ in range(max_new_tokens):
            out = self.forward(x)
            probs = torch.softmax(out[-1, :], dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            tokens.append(next_token.item())
            
            if next_token == 59 or len(tokens) > 32: # eos and max context length
                break
            
            x = torch.tensor(tokens)
        return tokens

## TEXT DATASET

In [14]:
pad_id = vocab[" "]

class TextDataset(Dataset):
    def __init__(self, token_ids:list, context_length:int, stride:int):
        super().__init__()

        self.inputs = []
        self.targets = []

        for i in range(0, len(token_ids) - context_length, stride):
            input_chunk = token_ids[i : i+context_length]
            target_chunk = token_ids[i+1 : i+context_length+1]

            # padding
            input_chunk = input_chunk + [pad_id] * (context_length - len(input_chunk))
            target_chunk = target_chunk + [pad_id] * (context_length - len(target_chunk))

            # truncate
            input_chunk = input_chunk[:context_length]
            target_chunk = target_chunk[:context_length]

            # tensor -> multi dimensional list
            self.inputs.append(torch.tensor(input_chunk, dtype=torch.long)) # Embedding layer → torch.long ister
            self.targets.append(torch.tensor(target_chunk, dtype=torch.long))

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]

## 3 - TRAINING

In [15]:
tokenizer = Tokenizer(vocab)
tokens = tokenizer.encode(text)
tokens

[84,
 161,
 56,
 161,
 4,
 161,
 86,
 161,
 112,
 161,
 111,
 161,
 147,
 161,
 136,
 161,
 3,
 161,
 85,
 161,
 1,
 161,
 124,
 161,
 72,
 0,
 161,
 134,
 161,
 78,
 161,
 82,
 161,
 159,
 161,
 104,
 161,
 160,
 161,
 32,
 161,
 139,
 0,
 161,
 135,
 161,
 60,
 161,
 151,
 161,
 100,
 161,
 31,
 161,
 147,
 161,
 141,
 161,
 90,
 161,
 25,
 0,
 161,
 134,
 161,
 66,
 161,
 80,
 161,
 19,
 161,
 120,
 161,
 117,
 161,
 53,
 161,
 129,
 161,
 75,
 161,
 61,
 161,
 62,
 161,
 71,
 0,
 161,
 130,
 161,
 113,
 161,
 68,
 161,
 141,
 161,
 134,
 161,
 107,
 161,
 72,
 0,
 161,
 11,
 161,
 106,
 161,
 143,
 161,
 48,
 161,
 134,
 161,
 109,
 161,
 30,
 161,
 141,
 161,
 15,
 161,
 32,
 161,
 149,
 0,
 161,
 132,
 161,
 142,
 161,
 137,
 161,
 110,
 161,
 55,
 161,
 65,
 161,
 49,
 0,
 161,
 6,
 161,
 73,
 161,
 76,
 161,
 17,
 0,
 161,
 7,
 161,
 153,
 161,
 57,
 161,
 91,
 161,
 158,
 161,
 148,
 0,
 161,
 21,
 161,
 157,
 161,
 7,
 161,
 154,
 161,
 70,
 0,
 161,
 13,
 161,
 156,
 161,
 1

In [16]:
embedding_dim = 32
context_length = 12
vocab_size = len(tokenizer.vocab)

torch.manual_seed(0)  # Reproducibility için rastgele tohum belirleme

model = MasterLLMModel(vocab_size, embedding_dim, context_length, dropout_rate=0.5, num_heads=2, num_layers=3)
model

MasterLLMModel(
  (embedding): Embedding(
    (embedding): Embedding(163, 32)
  )
  (layers): Sequential(
    (0): DecoderBlock(
      (self_attention): MultiHeadAttention(
        (multi_head_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
        )
        (projection): Linear(in_features=32, out_features=32, bias=True)
      )
      (norm1): LayerNorm()
      (mlp): MLP(
        (gate_proj): Linear(in_features=32, out_features=32, bias=True)
        (up_proj): Linear(in_features=32, out_features=32, bias=True)
        (down_proj): Linear(in_features=32, out_features=32, bias=True)
        (gelu): GELU()
      )
      (norm2): LayerNorm()
    )
    (1): DecoderBlock(
      (self_attention): MultiHeadAttention(
        (multi_head_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
        )
        (projection): Linear(in_feat

In [17]:
stride = 2

dataset = TextDataset(tokens, context_length, stride)
len(dataset.inputs), len(dataset.targets)

(205, 205)

In [18]:
out0 = model(dataset.inputs[0])
out0.shape

torch.Size([12, 163])

## LOSS FUNCTION - CrossEntropyLoss

In [19]:
loss_fn = nn.CrossEntropyLoss()
loss = loss_fn(out0, dataset.targets[0])
loss

tensor(4.7166, grad_fn=<NllLossBackward0>)

## OPTIMIZER - AdamW

In [20]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [21]:
epoch = 500

for epoch in range(epoch):
    total_loss = 0
    for input_, target in dataset:
        pred = model(input_)
        loss = loss_fn(pred, target)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
    avg_loss = total_loss / len(dataset)
    print(f"Epoch {epoch + 1} Loss: {loss.item()} Average Loss: {avg_loss}") 
        

Epoch 1 Loss: 2.6079323291778564 Average Loss: 2.832050931744459
Epoch 2 Loss: 2.352301836013794 Average Loss: 2.479521927019445
Epoch 3 Loss: 2.436436891555786 Average Loss: 2.281696034059292
Epoch 4 Loss: 2.2793798446655273 Average Loss: 2.2057758057989725
Epoch 5 Loss: 2.1747593879699707 Average Loss: 2.0757450167725726
Epoch 6 Loss: 2.1898000240325928 Average Loss: 1.9558254108196351
Epoch 7 Loss: 2.246631383895874 Average Loss: 1.8921991383157126
Epoch 8 Loss: 2.191065788269043 Average Loss: 1.8473656508980727
Epoch 9 Loss: 2.2283077239990234 Average Loss: 1.8500280630297776
Epoch 10 Loss: 2.755199670791626 Average Loss: 1.8879931915097121
Epoch 11 Loss: 2.2491819858551025 Average Loss: 1.9421470444376876
Epoch 12 Loss: 2.7812461853027344 Average Loss: 1.9784919814365667
Epoch 13 Loss: 2.6682260036468506 Average Loss: 2.245233061836987
Epoch 14 Loss: 2.7451722621917725 Average Loss: 2.1970950894239474
Epoch 15 Loss: 2.386631965637207 Average Loss: 2.2113191587168997
Epoch 16 Loss:

## 4 - TEST

In [29]:
test_tokens = tokenizer.encode("mh370 yolcu")
out = model.generate(test_tokens,10)
tokenizer.decode(out)

'mh370 yolcu uçağı 8 mart 2014 tarihinde'

## SAVE MODEL

In [ ]:
torch.save(model.state_dict(), "model.pth")

## LOAD MODEL

In [ ]:
loaded_model = model = MasterLLMModel(vocab_size=len(vocab), embedding_dim=embedding_dim, context_length=context_length, dropout_rate=0.5, num_heads=2, num_layers=3)
loaded_model.load_state_dict(torch.load("model.pth"))
loaded_model